In [1]:
from datasets import load_dataset
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
import gc
from torch.utils.data import DataLoader

In [2]:
import gc
gc.collect()

20

In [ ]:
import json

In [ ]:
file_path = '/home/hayoung/ss/model.safetensors.index.json'

with open(file_path, 'r', encoding='utf-8') as json_file:
    data_dict = json.load(json_file)

# 딕셔너리 내용 출력 (선택 사항)
print(data_dict)

In [3]:
torch.random.manual_seed(0)
model_id = "microsoft/Phi-3-medium-4k-instruct"
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="cpu", 
    torch_dtype="auto", 
    trust_remote_code=True, 
    cache_dir = '/nas/user/hayoung'
)
tokenizer = AutoTokenizer.from_pretrained(model_id)

config.json:   0%|          | 0.00/934 [00:00<?, ?B/s]

configuration_phi3.py:   0%|          | 0.00/10.4k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/microsoft/Phi-3-medium-4k-instruct:
- configuration_phi3.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


modeling_phi3.py:   0%|          | 0.00/73.8k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/microsoft/Phi-3-medium-4k-instruct:
- modeling_phi3.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


...

model-00004-of-00006.safetensors:   0%|          | 0.00/4.77G [00:00<?, ?B/s]

model-00005-of-00006.safetensors:   0%|          | 0.00/4.77G [00:00<?, ?B/s]

model-00006-of-00006.safetensors:   0%|          | 0.00/3.61G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/6 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/172 [00:00<?, ?B/s]

In [12]:
dataset = load_dataset("allenai/openbookqa", split='validation')

In [11]:
model

Phi3SmallForCausalLM(
  (model): Phi3SmallModel(
    (embed_tokens): Embedding(100352, 4096)
    (embedding_dropout): Dropout(p=0.1, inplace=False)
    (layers): ModuleList(
      (0): Phi3SmallDecoderLayer(
        (self_attn): Phi3SmallSelfAttention(
          (query_key_value): Linear(in_features=4096, out_features=6144, bias=True)
          (dense): Linear(in_features=4096, out_features=4096, bias=True)
          (_blocksparse_layer): BlockSparseAttentionLayer()
          (rotary_emb): RotaryEmbedding()
        )
        (mlp): Phi3SmallMLP(
          (up_proj): Linear(in_features=4096, out_features=28672, bias=True)
          (down_proj): Linear(in_features=14336, out_features=4096, bias=True)
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (input_layernorm): LayerNorm((4096,), eps=1e-05, elementwise_affine=True)
        (post_attention_layernorm): LayerNorm((4096,), eps=1e-05, elementwise_affine=True)
      )
      (1): Phi3SmallDecoderLayer(
        (self_at

In [13]:
dataset

Dataset({
    features: ['id', 'question_stem', 'choices', 'answerKey'],
    num_rows: 500
})

In [29]:
test = dataset.select(range(20))
test

Dataset({
    features: ['id', 'question_stem', 'choices', 'answerKey'],
    num_rows: 20
})

In [64]:
max_length = 100
prefix = "\nRead the question and answer the following sentence in given multiple choice.\nAnswer only the sentence you chose. Never include a question and other word in your answer.\n\nquestion: "

model_inputs = []

def preprocess(data):
    
    for i in range(len(data['question_stem'])):
        chat_dict = {
            "messages" :[
                {
                    "role" : "user",
                    "content" : prefix + data['question_stem'][i] + "\nchoices: ["
                }
            ],
            "answer" : data['answerKey'][i]
        }
        
        for j in range(4):
            chat_dict['messages'][0]['content'] += "\'" + data['choices'][i]['text'][j]
            if j < 3:
                chat_dict['messages'][0]['content'] += "\', "
            else:
                chat_dict['messages'][0]['content'] += "\']\n"
    
    model_inputs.append(chat_dict)
    

In [66]:
test.map(preprocess, batched=True, batch_size=5)
model_inputs[0:2]

Map:   0%|          | 0/20 [00:00<?, ? examples/s]

[{'messages': [{'role': 'user',
    'content': "\nRead the question and answer the following sentence in given multiple choice.\nAnswer only the sentence you chose. Never include a question and other word in your answer.\n\nquestion: An example of lots kinetic energy would be\nchoices: ['Drinking a cold glass of water', 'A snail moving across the sidewalk', 'sitting without moving anywhere', 'An aircraft taking a trip']\n"}],
  'answer': 'D'},
 {'messages': [{'role': 'user',
    'content': "\nRead the question and answer the following sentence in given multiple choice.\nAnswer only the sentence you chose. Never include a question and other word in your answer.\n\nquestion: A body may find its temperature to be lowered after\nchoices: ['water is heated up', 'fluid spreads from pores', 'the air becomes arid', 'the sky stays bright']\n"}],
  'answer': 'B'}]

In [68]:
tokenized_messages = tokenizer.apply_chat_template(model_inputs['messages'], tokenize=False, return_tensors="pt")
tokenized_messages

TypeError: list indices must be integers or slices, not str

In [52]:
tokenized_messages.to('cuda')

tensor([[32000]], device='cuda:0')

In [23]:
decoded_labels = [tokenizer.decode(label_ids, skip_special_tokens=True) for label_ids in model_inputs['labels']]
decoded_labels

['B',
 'A',
 'C',
 'C',
 'C',
 'C',
 'C',
 'B',
 'D',
 'B',
 'C',
 'B',
 'C',
 'A',
 'C',
 'D',
 'C',
 'C',
 'A',
 'B']

In [24]:
allowed_tokens = ['A', 'B', 'C', 'D']
allowed_token_ids = tokenizer.convert_tokens_to_ids(allowed_tokens)

In [53]:
with torch.no_grad():
    outputs = model(input_ids=model_inputs['input_ids'], attention_mask=model_inputs['attention_mask'])
print(outputs.logits.shape)
print(outputs.logits[:,-1,:].shape)
processed = torch.nn.functional.softmax(outputs.logits[:,-1,:], dim=1)
print(processed.shape)
generated = torch.argmax(processed,dim=1)
print(generated.shape)
decoded_outputs = [tokenizer.decode(answer, skip_special_tokens=True) for answer in generated]
print(decoded_outputs)

torch.Size([20, 100, 32064])
torch.Size([20, 32064])
torch.Size([20, 32064])
torch.Size([20])
['A', 'only', 'and', ',', 'from', 'ose', 'A', '.', '\n', 'one', 'one', ',', 'C', 'd', 'C', ',', 'answer', 'Cho', ',', ',']


In [28]:
with torch.no_grad():
    outputs = model(input_ids=model_inputs['input_ids'], attention_mask=model_inputs['attention_mask']
                            )


suffix = "answer:"

generated_output = [tokenizer.decode(output, skip_special_tokens=True) for output in outputs]
position = [output.find(suffix) for output in generated_output]
print(position)
post = []
for i, pos in enumerate(position):
    if pos == -1:
        post.append(generated_output[i])
    else:
        post.append(generated_output[i][pos+7:])
print(post)

TypeError: argument 'ids': Can't extract `str` to `Vec`

In [ ]:
class CustomedPipeline():
    def __init__(
            self,
            config,
            model_id = "microsoft/Phi-3-mini-4k-instruct"
            device = "cuda"
        ):
        self.config = config
        self.tokenizer = AutoTokenizer.from_pretrained(model_id)
        self.model =  CustomedPhi3ForCausalLM(self.tokenizer, self.config)
        
    
    def forward(self, model_inputs, max_length = 500):
        input_ids = model_inputs['input_ids']
        attention_mask = model_inputs['attention_mask']
        prompt_len = model_inputs['prompts']

        generated_sequence = self.model.generate(input_ids=input_ids, attention_mask=attention_mask,max_length=max_length)
        return {"generated_sequence": generated_sequence, "prompt_len" :prompt_len}

    def postprocess(self, model_outputs, clean_up_tokenization_spaces=True):
        generated_sequence = model_outputs["generated_sequence"]
        prompt_len = model_outputs["prompt_len"]
        
        result = []
        
        for i, text in enumerate(generated_sequence):
            eos_pos = (text == self.tokenizer.eos_token_id).nonzero(as_tuple=True)[0]
  
            if len(eos_pos) > 0:
                eos_after_prompt = next((pos.item() for pos in eos_pos if pos.item() > prompt_len), None)

                if eos_after_prompt is not None:
                    text = text[prompt_len:eos_after_prompt-1]
                else:
                    text = text[prompt_len:]
            else:
                text = text[prompt_len:]
                
            #decoded_text = self.tokenizer.decode(text, skip_special_tokens=True)
            decoded_text = self.tokenizer.decode(text)
            result.append([{'generated':decoded_text}])

        return result

In [ ]:
gc.collect()